In [1]:
import numpy as np
import sys

In [2]:
# Prepare
config = {}
config['topk'] = 5
config['nms_th'] = 0.05
#config['nms_th'] = 0.0001
config['conf_th'] = -1


In [3]:
pbb = np.load("./0de72529c30fe642bc60dcb75c87f6bd_pbb.npy")
pbb = pbb[pbb[:, 0] > config['conf_th']]

pbb.shape

(1228800, 5)

In [4]:
pbb[0][1:5]

array([  3.42642346,   1.43374363,   2.20192143,  10.84307154])

In [5]:
def iou(box0, box1):
    r0 = box0[3] / 2
    s0 = box0[:3] - r0
    e0 = box0[:3] + r0
    r1 = box1[3] / 2
    s1 = box1[:3] - r1
    e1 = box1[:3] + r1
    overlap = []
    
    for i in range(len(s0)):
        overlap.append(max(0, min(e0[i], e1[i]) - max(s0[i], s1[i])))

    intersection = overlap[0] * overlap[1] * overlap[2]
    union = box0[3] * box0[3] * box0[3] + box1[3] * box1[3] * box1[3] - intersection
    # print("intersection:{} and union:{}".format(intersection, union))
    return intersection / union

In [6]:
def nms(output, nms_th, valid_size=None):
    if len(output) == 0:
        return output

    output = output[np.argsort(-output[:, 0])]
    bboxes = [output[0]]
    for i in np.arange(1, len(output)):
        sys.stdout.flush()
        bbox = output[i]
        flag = 1
        for j in range(len(bboxes)):
            if iou(bbox[1:5], bboxes[j][1:5]) >= nms_th:
                flag = -1
                break
        if flag == 1:
            bboxes.append(bbox)
            if valid_size is not None and len(bboxes) >= valid_size:
                break

    bboxes = np.asarray(bboxes, np.float32)
    return bboxes

In [7]:
%timeit nms(pbb, config['nms_th'], config['topk']*10)

1 loop, best of 3: 1.83 s per loop


In [8]:
%load_ext Cython

In [9]:
%%cython --annotate
import numpy as np

def iou_cython(box0, box1):
    cdef float r0 = box0[3] / 2
    s0 = box0[:3] - r0
    e0 = box0[:3] + r0
    cdef float r1 = box1[3] / 2
    s1 = box1[:3] - r1
    e1 = box1[:3] + r1
    overlap = []
    for i in range(len(s0)):
        overlap.append(max(0, min(e0[i], e1[i]) - max(s0[i], s1[i])))

    cdef float intersection = overlap[0] * overlap[1] * overlap[2]
    cdef float union = box0[3] * box0[3] * box0[3] + box1[3] * box1[3] * box1[3] - intersection
    # print("intersection:{} and union:{}".format(intersection, union))
    return intersection / union

def nms_cython(output, nms_th, valid_size=None):
    if len(output) == 0:
        return output

    output = output[np.argsort(-output[:, 0])] 
    bboxes = [output[0]]
    
    cdef int output_len = output.shape[0]
    cdef int flag = 1
    
    for i in range(output_len):
        bbox = output[i]
        flag = 1
        for j in range(len(bboxes)):
            if iou_cython(bbox[1:5], bboxes[j][1:5]) >= nms_th:
                flag = -1
                break
        if flag == 1:
            bboxes.append(bbox)
            if valid_size is not None and len(bboxes) >= valid_size:
                break

    bboxes = np.asarray(bboxes, np.float32)
    return bboxes



In [11]:
%timeit nms_cython(pbb, config['nms_th'], config['topk']*10)

1 loop, best of 3: 603 ms per loop


In [ ]:
nms_cython(pbb, config['nms_th'], config['topk']*1000)